# Crime mapping, visualization and predictive analysis

This notebook demonstrates techniques for analyzing data that can be used to more efficiently manage and distribute police resources, with a goal of decreasing crime. The workflow involves fetching and preparing big data for analysis and visualization using hotspots, geographic aggregation of data, enrichment using demographic variables and Support Vector Classification (SVC) using SciKit-learn. 

Crime is a complex interaction of many processes that this notebook doesn't fully account for. Machine learning algorithms are also susceptible to unintended biases that a comprehensive solution would require careful planning to avoid. This notebooks provides a simple demonstration of how GIS technology and machine learning can be used to identify crime prone areas and build predictive policing models using publicly accessible data. 

# Data Preparation

The Hoston Police department shares historical crime statistics at http://www.houstontx.gov/police/cs/crime-stats-archives.htm that we'll be using for our analysis.
<img src="../../static/img/hpd.png" width="750"/>

## Fetch data

In [1]:
import pandas as pd
import calendar

In [20]:
combined_df = pd.DataFrame()

for year in range(10, 17):
    for month in range(1, 13):
        url = 'http://www.houstontx.gov/police/cs/xls/{0}{1}.xls'.format(
            calendar.month_name[month].lower()[:3], year)
        df = pd.read_excel(url)
        combined_df = combined_df.append(df, ignore_index=True)

/anaconda3/envs/geosaurus/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


WARNING *** file size (2752234) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [21]:
# archive the data so we don't need to re-fetch it:
combined_df.to_csv(r'./data/crime_df.csv', index=False)

## Data cleanup

In [13]:
crimes_df = pd.read_csv(r'./data/crime_df.csv', 
                        parse_dates=[7], low_memory=False)

In [14]:
crimes_df.shape

(886604, 17)

In [15]:
crimes_df.head(3)

,# Of,# Of Offenses,# Offenses,# offenses,Beat,Block Range,BlockRange,Date,Field11,Hour,Offense Type,Premise,Street Name,StreetName,Suffix,Type,Unnamed: 1
0,NaN,1.0,NaN,NaN,11H30,3000-3099,NaN,2010-01-08,NaN,22.0,Murder,20R,BROADWAY,NaN,-,ST,NaN
1,NaN,1.0,NaN,NaN,10H50,2800-2899,NaN,2010-01-17,NaN,18.0,Murder,18C,MCGOWEN,NaN,-,ST,NaN
2,NaN,1.0,NaN,NaN,15E30,9600-9699,NaN,2010-01-01,NaN,0.0,Murder,18A,MARLIVE,NaN,-,LN,NaN


In [16]:
crimes_df.loc[103082]

# Of             NaN
# Of Offenses    NaN
# Offenses       NaN
# offenses       NaN
Beat             NaN
Block Range      NaN
BlockRange       NaN
Date             NaT
Field11          NaN
Hour             NaN
Offense Type     NaN
Premise          NaN
Street Name      NaN
StreetName       NaN
Suffix           NaN
Type             NaN
Unnamed: 1       NaN
Name: 103082, dtype: object

### Data wrangling

In [17]:
crimes_df_nona1 = crimes_df.dropna(axis=0, how='all')

In [18]:
crimes_df_nona1.shape

(886603, 17)

In [19]:
crimes_df_nona1['Address'] = crimes_df_nona1.HouseNo.astype(int).map(str) + ' ' + \
                       crimes_df_nona1.Suffix.map(str) + ' ' + \
                       crimes_df_nona1.StreetName.map(str) + ' ' + \
                       crimes_df_nona1.Type.map(str) + ', Houston, TX'

crimes_df_nona1.dropna(axis=0, how='Address', inplace=True)
crimes_df_nona1.shape

AttributeError: 'DataFrame' object has no attribute 'HouseNo'

In [20]:
# drop empty rows and replace '-' with ''
crimes_df.dropna(axis=0, how='all', inplace=True)
crimes_df = crimes_df.replace('-', '')

# offenses are spread across four columns with different name. Similar for BlockRange and StreetName. Combine the columns:
crimes_df['Offenses'] = (crimes_df['# Of'].combine_first( \
                         crimes_df['# Of Offenses']).combine_first( \
                         crimes_df['# Offenses']).combine_first( \
                         crimes_df['# offenses'])).astype(int)

crimes_df['BlockRange'] = crimes_df['BlockRange'].combine_first( \
                          crimes_df['Block Range'])

crimes_df['StreetName'] = crimes_df['StreetName'].combine_first( \
                          crimes_df['Street Name'])

# extract the approximate (mid) address from the block range 
# and construct an address column using it and the street suffix, name and type:
houseno = crimes_df.BlockRange.str.extract('(?P<start_addr>\d+)-(?P<end_addr>\d+)', expand=False)
crimes_df['HouseNo'] = (houseno.start_addr.astype(float) + houseno.end_addr.astype(float) + 1.0) / 2
crimes_df['HouseNo'] = crimes_df['HouseNo'].fillna(50.0)

crimes_df['Address'] = crimes_df.HouseNo.astype(int).map(str) + ' ' + \
                       crimes_df.Suffix.map(str) + ' ' + \
                       crimes_df.StreetName.map(str) + ' ' + \
                       crimes_df.Type.map(str) + ', Houston, TX'
            
# the hour column contains strings (with leading quotes), floats and int as well as empty strings. Convert them all to ints:
crimes_df['Hour'] = crimes_df['Hour'].apply(lambda x : int(float(x.replace("'", ""))) if type(x) == str else int(float(x)))

# create a new column for Day of the week, useful for analysis:
crimes_df['DayOfWeek'] = crimes_df.Date.dt.weekday_name

# rename Offense Type to Category
crimes_df = crimes_df.rename(columns={'Offense Type' : 'Category'})

In [21]:
crimes_df.shape

(886603, 21)

In [24]:
crimes_df.dropna(axis=0, subset=['Address']).shape

(886603, 21)

In [6]:
crimes_df = crimes_df.reset_index(drop=True)

In [7]:
crimes_df.loc[103082]

# Of                                     NaN
# Of Offenses                              1
# Offenses                               NaN
# offenses                               NaN
Beat                                   17E10
Block Range                        7100-7199
BlockRange                         7100-7199
Date                     2010-10-18 00:00:00
Field11                                  NaN
Hour                                      21
Category                              Murder
Premise                                  05D
Street Name                            ALDER
StreetName                             ALDER
Suffix                                      
Type                                      DR
Unnamed: 1                               NaN
Offenses                                   1
HouseNo                                 7150
Address          7150  ALDER DR, Houston, TX
DayOfWeek                             Monday
Name: 103082, dtype: object

### Cleaned up data

In [8]:
analysis_df = crimes_df[['Date', 'Hour', 'DayOfWeek', 'Category', 'Offenses', 'Beat', 'Address']]
analysis_df.head()

,Date,Hour,DayOfWeek,Category,Offenses,Beat,Address
0,2010-01-08,22,Friday,Murder,1,11H30,"3050 BROADWAY ST, Houston, TX"
1,2010-01-17,18,Sunday,Murder,1,10H50,"2850 MCGOWEN ST, Houston, TX"
2,2010-01-01,0,Friday,Murder,1,15E30,"9650 MARLIVE LN, Houston, TX"
3,2010-01-09,14,Saturday,Murder,1,5F20,"9050 LONG POINT RD, Houston, TX"
4,2010-01-18,14,Monday,Murder,1,10H40,"4350 GARROTT ST, Houston, TX"


In [9]:
houston_processed = r'./data/houston_processed.csv'
analysis_df.to_csv(houston_processed, index_label='id')

In [10]:
analysis_df.shape

(886603, 7)

## Convert to Spatial dataframe with geocoded addresses

Spatial Dataframes let you work with spatial data as if it were a Pandas dataframe. It adds geospatial operations to Pandas as well as the ability to read and write shapefiles and file geodatabases.

You can convert any Pandas dataframe with an address column to a SpatialDataFrame. Note that this operation geocodes all addresses within the dataframe and this will consume credits if your organization/portal is configured to use ArcGIS Online World Geocoder.

In [11]:
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# gis = GIS(profile="your_enterprise_profile") 
# gis = GIS("https://datascienceqa.esri.com/portal", "portaladmin", "esri.agp", verify_cert=False)
gis = GIS(profile="your_online_profile") 

In [12]:
# Note - uncomment the following line to geocode all addresses within the dataframe and create a SpatialDataFrame
# This will consume credits if your organization/portal is configured to use ArcGIS Online World Geocoder.

# spdf = SpatialDataFrame.from_df(houston_processed, 'Address')

spdf = pd.DataFrame.spatial.from_df(analysis_df, 'Address')
spdf.head()

# spdf.sr = arcgis.geometry.SpatialReference(4326)
# spdf.sr = SpatialReference({"wkid" : 4326})

/Users/yong9856/Documents/GitHub/geosaurus/src/arcgis/features/geo/_accessor.py:1957: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['SHAPE'] = geoms
/anaconda3/envs/geosaurus/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/yong9856/Documents/GitHub/geosaurus/src/arcgis/features/geo/_accessor.py:961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,Date,Hour,DayOfWeek,Category,Offenses,Beat,Address,SHAPE
0,2010-01-08,22,Friday,Murder,1,11H30,"3050 BROADWAY ST, Houston, TX","{""x"": -95.27787570099997, ""y"": 29.696461096000..."
1,2010-01-17,18,Sunday,Murder,1,10H50,"2850 MCGOWEN ST, Houston, TX","{""x"": -95.35841779399999, ""y"": 29.735879410000..."
2,2010-01-01,0,Friday,Murder,1,15E30,"9650 MARLIVE LN, Houston, TX","{""x"": -95.43742728899997, ""y"": 29.678459999000..."
3,2010-01-09,14,Saturday,Murder,1,5F20,"9050 LONG POINT RD, Houston, TX","{""x"": -95.51349431499995, ""y"": 29.800234248000..."
4,2010-01-18,14,Monday,Murder,1,10H40,"4350 GARROTT ST, Houston, TX","{""x"": -95.38550667799996, ""y"": 29.733398564000..."


## Pandas Dataframe to shapefiles or file geodatabase

In [ ]:
import os

for year in range(2010, 2017):
    start_date = str(year) + '-01-01'
    end_date = str(year) + '-12-31'
    
    mask = (spdf['Date'] > start_date) & (spdf['Date'] <= end_date)
    spdf_year = spdf.loc[mask]
    
    directory = r'./data/houstoncrime' + str(year) 
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    spdf_year.to_featureclass(out_location=directory,
                              out_name='Houston' + str(year) + '.shp')

In [ ]:
spdf.to_featureclass(out_location=r"./data/houston.gdb", out_name="crime")

## Connect the data to your GIS

#### Publish as hosted feature layer

In [ ]:
data = r'C:\xc\Presentations\Demos\Houston\shp\houston2016.zip' 
shpfile = gis.content.add({}, data)

In [ ]:
crime2016 = shpfile.publish()

In [ ]:
crime2016 = gis.content.search('houston2016', 'feature layer')[0]
crime2016

#### Attach to geoanalytics datastore as bigdata file share

In [ ]:
datastores = arcgis.geoanalytics.get_datastores()

In [ ]:
houston_bigdata = datastores.add_bigdata('Houston_crime', r'\\teton\atma_shared\datasets\Houstonshp')

In [ ]:
houston_yearly_bigdata = datastores.add_bigdata('Houston_crime_yearly', r'\\teton\atma_shared\datasets\HoustonCrime')

The datasets within the attached bigdata fileshare show up as Items and Layers and are available for analysis:

In [ ]:
items = gis.content.search("houston", "big data file share")
for item in items:
    display(item)

In [ ]:
houston_crime = items[0]
houston_yearly = items[1]

In [ ]:
houston_yearly.layers

# Mapping, Visualization and Analysis

Visualize crime layer using the Map widget and Smart Mapping:

In [ ]:
houston = arcgis.geocoding.geocode('Houston, TX')[0]

In [ ]:
crime2016 = gis.content.search('houston2016', 'feature layer')[0]

map1 = gis.map(houston, 12)
map1.add_layer(crime2016)

map2 = gis.map(houston, 12)
map2.add_layer(crime2016, {"renderer":"HeatmapRenderer", "opacity":0.75})

from ipywidgets import *

map1.layout=Layout(flex='1 1', padding='10px')
map2.layout=Layout(flex='1 1', padding='10px')

crimebox = HBox([map1, map2])
crimebox

## Exploratory data analysis

In [ ]:
from arcgis import SpatialDataFrame

df = SpatialDataFrame.from_featureclass(filename = r"C:\xc\Presentations\Demos\Houston\houston.gdb\crime")

In [ ]:
df.columns

What are the most comon crimes?

In [ ]:
# crimes by category

%matplotlib inline
import matplotlib.pyplot as plt

groups = df.groupby("Category")["Category"].count()
groups = groups.sort_values(ascending=False)
plt.figure()
groups.plot(kind='bar', title="Crimes by Category")

Does crime distribution change by the day of the week?

In [ ]:
fig, axs = plt.subplots(1,2)

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df_murder = df[df.Category == "Murder"]
gr_murder = df_murder.groupby("DayOfWeek")["Offenses"].sum()
gr_murder = gr_murder[weekdays]

df_burglary = df[df.Category == "Burglary"]
gr_burglary = df_burglary.groupby("DayOfWeek")["Offenses"].sum()
gr_burglary = gr_burglary[weekdays]

gr_murder.plot(kind="bar", title="Murder per weekday", ax=axs[0])
gr_burglary.plot(kind="bar", title="Burglary per weekday", ax=axs[1], figsize=(12, 5))

Which crimes occur during which part of the day?

In [ ]:
# thefts and burglaries by hour
fig, axs = plt.subplots(1,2)
hours = [x for x in range(24)]
df_theft = df[df.Category == "Murder"]
gr_theft = df_theft.groupby("Hour")["Category"].count()
gr_theft = gr_theft[hours]

df_burglary = df[df.Category == "Burglary"]
gr_burglary = df_burglary.groupby("Hour")["Category"].count()
gr_burglary = gr_burglary[hours]

gr_theft.plot(kind="bar", title="Murder per hour", ax=axs[0])
gr_burglary.plot(kind="bar", title="Burglary per hour", ax=axs[1], figsize=(20, 5))

## Big data analytics

GeoAnalytics provides a set of powerful tools for performing spatial analysis on big data. GeoAnalytics Tools are powered by your ArcGIS GeoAnalytics Server. ArcGIS GeoAnalytics Server distributes the analysis between multiple server nodes. By using distributed processing, you can process large datasets in less time.

### Find hot spots of crime

The “Find Hot Spots” tool can be used to identify statistically significant clusters of crime.

We can automate the creation of hotspots for all crime categories, across the years and create persisted information products in the form of web layers:

In [ ]:
from arcgis.geoanalytics.analyze_patterns import find_hot_spots
arcgis.env.process_spatial_reference=32611
arcgis.env.verbose = False

In [ ]:
for category in df.Category.unique()[:-1]:
    lyrid = 0
    for year in range(2010, 2017):
        output_name='Houston_' + category.replace(' ', '_') + '_Hotspot_' + str(year)
        print('Generating ' + output_name)
        layer = houston_yearly.layers[lyrid]
        layer.filter = "Category='{}'".format(category)
        
        find_hot_spots(layer, bin_size=0.5, bin_size_unit='Miles', 
                       neighborhood_distance=1, neighborhood_distance_unit='Miles', output_name=output_name)
        
        lyrid = lyrid + 1

##### Compare Burglary hot spots with auto-theft hot spots

In [ ]:
hotmap1 = gis.map(houston, 10)
hotmap1.add_layer(gis.content.search('Houston_Burglary_Hotspot_2016')[0])
hotmap2 = gis.map(houston, 10)
hotmap2.add_layer(gis.content.search('Houston_Auto_Theft_Hotspot_2016')[0])

hotmap1.layout=Layout(flex='1 1', padding='3px')
hotmap2.layout=Layout(flex='1 1', padding='3px')

items_layout = Layout(flex='1 1 auto', width='auto')

display(HBox([hotmap1, hotmap2]))
display(HBox(children=[Button(description='Burglary hot spots in 2016', layout=items_layout, button_style='danger'),
                       Button(description='Auto theft hot spots in 2016', layout=items_layout, button_style='danger')],
             layout=Layout(width='100%')))

##### Compare hot spots over time

In [ ]:
maps = []
labels = []
items_layout = Layout(flex='1 1 auto', width='auto')
for year in range(2014, 2017):
    layer = gis.content.search('Houston_Auto_Theft_Hotspot_'+str(year))[0]
    hotspotmap = gis.map(houston)
    hotspotmap.add_layer(layer)
    hotspotmap.layout=Layout(flex='1 1', padding='3px')
    maps.append(hotspotmap)
    hotspotmap.basemap='gray'
    labels.append(Button(description='Auto theft hot spots in ' + str(year), layout=items_layout, button_style='danger'))
    
layout=Layout(height='300px')
display(HBox([maps[0], maps[1], maps[2]], layout=layout))
display(HBox(children=labels, layout=Layout(width='100%')))

The hot spots in each category are strikingly similar across years. The hot spots of yesterday are the hotspots of tomorrow, and this information can be used to deploy officers and to identify areas in need of intervention.

## Aggregate crimes into police beats

In [ ]:
items = gis.content.search('Houston Police Beats', 'feature layer')
for item in items:
    display(item)

In [ ]:
police_beats = items[0]

In [ ]:
from arcgis.geoanalytics.summarize_data import aggregate_points
arcgis.env.verbose = True

aggregate_points(houston_crime.layers[0]._lyr_dict, 
                 polygon_layer=police_beats, output_name='Houston Aggregated Crimes')

In [ ]:
aggr_lyr = gis.content.search('Houston Aggregated Crimes')[0]
aggr_lyr

### Visualize crime aggregated by police beat

In [ ]:
beats_map = gis.map(houston, 11)
beats_map

In [ ]:
beats_map.basemap = 'gray'

In [ ]:
beats_map.add_layer(aggr_lyr, {
                        "renderer":"ClassedColorRenderer",
                        "field_name":"SUM_Offenses", 
                        "classificationMethod":'natural-breaks',
                        "numClasses":10,
                        "opacity":0.75
                      })

### Visualize aggregation results using plots

In [ ]:
aggr_df = aggr_lyr.layers[0].query().sdf

In [ ]:
aggr_df = aggr_df[['beats_0', 'SUM_Offenses']]
aggr_df.set_index('beats_0', inplace=True)
aggr_df.sort_values('SUM_Offenses', ascending=False).head(6)

In [ ]:
%matplotlib inline
aggr_df.sort_values('SUM_Offenses', ascending=False).head(30).plot(kind='bar')

In [ ]:
%matplotlib inline
aggr_df['SUM_Offenses'].hist()

# Predictive analysis using Machine Learning

### Feature engineering using data enrichment

In [ ]:
from arcgis.features import enrich_data

In [ ]:
enriched_houston = enrich_data.enrich_layer(police_beats, data_collections = ["KeyUSFacts"], 
                                            analysis_variables=["AtRisk.AVGHINC_CY","KeyUSFacts.DIVINDX_CY"])

In [ ]:
analysis_variables = [
    'TOTPOP_CY',  # 2016 Population: Total Population (Esri)
    'HHPOP_CY',   # 2016 Household Population (Esri)
    'FAMPOP_CY',  # 2016 Family Population (Esri)
    'DIVINDX_CY', # 2016 Diversity Index (Esri)
    'TOTHH_CY',   # 2016 Total Households (Esri)
    'AVGHHSZ_CY', # 2016 Average Household Size (Esri)

    'MALES_CY',   # 2016 Gender: Male Population (Esri)
    'FEMALES_CY', # 2016 Gender: Female Population (Esri)
    
    'MEDAGE_CY',  # 2016 Age: Median Age (Esri)
    
    'AVGFMSZ_CY', # 2016 Income: Average Family Size (Esri)
    'MEDHINC_CY', # 2016 Income: Median Household Income (Esri)
    'AVGHINC_CY', # 2016 Income: Average Household Income (Esri)
        
    'EDUCBASECY', # 2016 Educational Attainment Base (Esri)
    'NOHS_CY',    # 2016 Education: Less than 9th Grade (Esri)
    'SOMEHS_CY',  # 2016 Education: 9-12th Grade/No Diploma (Esri)
    'HSGRAD_CY',  # 2016 Education: High School Diploma (Esri)
    'GED_CY',     # 2016 Education: GED/Alternative Credential (Esri)
    'SMCOLL_CY',  # 2016 Education: Some College/No Degree (Esri)
    'ASSCDEG_CY', # 2016 Education: Associate's Degree (Esri)
    'BACHDEG_CY', # 2016 Education: Bachelor's Degree (Esri)
]

In [ ]:
enriched_crime = enrich_data.enrich_layer(police_beats,
                                          analysis_variables=analysis_variables)

In [ ]:
enriched_df = enriched_crime.query().sdf
enriched_df.head()

In [ ]:
enriched_df.columns

In [ ]:
edf = enriched_df[['beats_0'] + analysis_variables]

### Merge with original crime dataframe

In [ ]:
merged_df = df.merge(edf, left_on='Beat', right_on='beats_0')
merged_df.columns

### Apply machine learning using Scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import numpy as np

In [ ]:
df = merged_df
df.Category.unique()

In [ ]:
df = df[df.Category != '1']
df = df[df.DayOfWeek != '']

offence_encoder = preprocessing.LabelEncoder()
offences = offence_encoder.fit_transform(df['Category'])

hour = pd.get_dummies(df.Hour, prefix="hour")
day = pd.get_dummies(df.DayOfWeek, prefix="day")
beats = pd.get_dummies(df.Beat, prefix="beats")

In [ ]:
offence_encoder = preprocessing.LabelEncoder()
offences = offence_encoder.fit_transform(df['Category'])

In [ ]:
data = pd.concat([hour, day, beats], axis=1)

In [ ]:
data['Violent'] = df['Category'].apply(lambda x : 1 if x in ['Murder', 'Rape', 'Aggravated Assault'] else 0)

### Test SVC classifier on subset of data

In [ ]:
data = data[:1000]

In [ ]:
training, validation = train_test_split(data, train_size=.75)

In [ ]:
features = list(training.columns)[:-1]

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(training[features], training.Violent)

In [ ]:
svc.score(training[features], training.Violent)

In [ ]:
svc.score(validation[features], validation.Violent)

### Run model using entire data

In [ ]:
data = pd.concat([hour, day, beats], axis=1)
data['Violent'] = df['Category'].apply(lambda x : 1 if x in ['Murder', 'Rape', 'Aggravated Assault'] else 0)

In [ ]:
training, validation = train_test_split(data, train_size=.75)

In [ ]:
features = list(training.columns)[:-1]

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(training[features], training.Violent)

### Check accuracy of SVC classifier

In [ ]:
svc.score(training[features], training.Violent)

In [ ]:
svc.score(validation[features], validation.Violent)